# 条件随机场（CRF）

作者：杨岱川

时间：2019年10月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

## 条件随机场简介

条件随机场（Conditional random field, 简称CRF）是一种常用于模式识别和机器学习的统计建模方法，可以用于结构预测等。这是一种给定一组输入随机变量条件下，另一组输出随机变量的条件概率分布模型。本章主要讨论将条件随机场用于标注问题的应用，因此主要讲述线性链（linear chain）条件随机场，这时，问题变成了由输入序列对输出序列预测的判别模型。

李航《统计学习方法（第二版）》中对于CRF的描述过于理论，直接阅读会比较吃力，本章节的目标是对书中难点进行必要补充，让CRF知识点更容易理解。

## 应用背景

### 简单例子

参考阅读[Introduction to Conditional Random Fields](http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/)

不同于《统计学习方法（第二版）》开篇就上公式，我们先通过例子或者例题来理解知识点会更有效：

>假设我们有小明一天中的一些生活照，我们希望用它所代表的活动（吃饭 or 睡觉 or 开车）来标注每张照片，应该怎么做？

最简单的方法就是构建一个图片分类器，比如卷积神经网络或者是别的什么方法等，然后对各种标记的图像进行分类。比如早上6点拍摄的深色图片往往表示小明在睡觉；霓虹灯闪烁的图片可能是小明在跳舞；而汽车图片则表示小明正在开车。

但是如果我们忽略了**顺序**信息，我们很可能会遇到一些问题。

>比如有一张小明**张嘴**的特写照片，那你应该怎么标注？张嘴有可能是“吃饭”，也有可能是“正在唱歌”——但是如果我们手里面有**前一时刻**的照片，比如小明正在做饭或者吃饭，那么这张“张嘴”照片大概率也是在吃饭；而如果前一时刻的照片显示小明拿着话筒在跳舞，那么这张“张嘴”的照片则大概率是在唱歌。

所以，为了让这个照片分类器有更好的表现，在给这一系列照片分类（标注）的时候，我们如果能把相邻时刻（前后均可）照片的标签信息考虑进来就更好啦！这就是CRF的作用。

### 词性标注

词性标注是自然语言处理（简称NLP）中非常常见的问题，就是给一个句子中每一个词标注其词性。比如：

>原句：Bob drank coffee at Starbucks
>
>标注后：Bob (NOUN) drank (VERB) coffee (NOUN) at (PREPOSITION) Starbucks (NOUN)

我们需要针对性地建立一个CRF来标记句子。

## 条件随机场的定义与形式

###  条件随机场的定义

### 线性链条件随机场的定义

### 条件随机场的参数化形式